In [ ]:
import datetime
import json
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

def getEventIndex(events, item):
  for i, ev in enumerate(events):
    if ev.get('description')!=None and item["description"] == ev["description"]:
      return i
  return -1

# Service Account Keyのロード
with open('credentials.json') as f:
    creds_data = json.load(f)

creds = Credentials.from_service_account_info(creds_data)
calendar_id = "do8vqrsgjkl70t0jvg3mbr9dh0@group.calendar.google.com"

# Google Calendar APIのserviceを作成
service = build('calendar', 'v3', credentials=creds)

event_result = (
    service.events()
    .list(
        calendarId=calendar_id,
        #                                     timeMin=st.isoformat()+'T00:00:00Z',
        #                                     timeMax=ed.isoformat()+'T00:00:00Z',
        singleEvents=True,
        orderBy="startTime",
    )
    .execute()
)
events = event_result.get("items", [])

def timeformat(t):            
    ret = t.get("dateTime", t.get("date"))
    return datetime.datetime.strptime(ret[:-6], "%Y-%m-%dT%H:%M:%S")
def timeformatNozone(t):            
    ret = t.get("dateTime", t.get("date"))
    return datetime.datetime.strptime(ret, "%Y-%m-%dT%H:%M:%S")

# Display all events
if not events:
    print("No upcoming events found.")
for i, event in enumerate(events):
    #start = event["start"].get("dateTime", event["start"].get("date"))
    #start = datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
    #start = event["start"]
    start = timeformat(event["start"])
    print("{:02}".format(i), start, event["summary"])
    if i > 10:
        break
print("Found items on Gcal",len(events))

In [ ]:
def getEventIndex(events, item):
    for i, event in enumerate(events):
        if item["description"] == event["description"]:
            if item["summary"]!=event["summary"]:
                return i, False
            if timeformatNozone(item["start"])!=timeformat(event["start"]):
                return i, False
            if timeformatNozone(item["end"])!=timeformat(event["end"]):
                return i, False
            if item.get("colorId")!=event.get("colorId"):
                return i, False
            return i, True
    return -1, False
            
f = open('events.json','r')
items = json.load(f)
f.close()
print(len(items),"loaded")

for i, item in enumerate(items):
    index, same = getEventIndex(events,item)
    #print(i,(item["start"]),item["summary"])
    if index < 0:
        print(i,"insert",timeformatNozone(item["start"]),item["summary"])
        service.events().insert(
                calendarId=calendar_id,
                body=item).execute()
    else:
        event = events.pop(index)
        #if same:
        #    print(i,"skip",timeformatNozone(item["start"]),item["summary"])
        if not same:
            print(i,"update",timeformat(event["start"]),event["summary"],"->", timeformatNozone(item["start"]),item["summary"])
            service.events().update(
                    calendarId=calendar_id, 
                    eventId=event["id"], 
                    body=item
                ).execute()

for event in events:
    print("delete",timeformat(event["start"]),event["summary"])
    service.events().delete(
        calendarId=calendar_id, 
        eventId=event["id"]).execute()

print("done")